In [ ]:
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
from plottable import ColumnDefinition, Table
from plottable.cmap import normed_cmap
from plottable.plots import image
import os
from urllib.request import urlopen
import matplotlib.font_manager as fm
from mplsoccer import FontManager
from PIL import Image
from highlight_text import fig_text, ax_text

In [ ]:
# Extract data from championship
info_brasileirao = pd.read_html('https://fbref.com/en/comps/24/Serie-A-Stats')[0]
info_brasileirao

In [ ]:
# Rename column to portuguese
columns_classification = {
    "Rk": "Cl",
    "Squad": "Equipe",
    "MP": "JG",
    "W": "V",
    "D": "E",
    "L": "D",
    "GF": "GP",
    "GA": "GC",
    "GD": "SG",
    "Pts": "Pts",
    "Pts/MP": "Pts/PPJ",
    "xG": "xG",
    "xGA": "xGC",
    "xGD": "xSG",
    "xGD/90": "xSG/90",
}

info_brasileirao = info_brasileirao[list(columns_classification.keys())].rename(columns=columns_classification)

In [ ]:
shields = pd.read_csv("/home/ericsonmatheus/personal/analysis_football/src/layers/1.raw/shields/clubs_id.csv", delimiter=";")

In [ ]:
brasileirao = pd.merge(info_brasileirao, shields, left_on="Equipe", right_on="team", how="left")

In [ ]:
from functions import get_shields

brasileirao["shield"] = brasileirao["shield_id"].apply(get_shields)


In [ ]:
# Filter columns to show
brasileirao[["xG", "xGC", "xSG", "xSG/90"]] = brasileirao[["xG", "xGC", "xSG", "xSG/90"]].astype(float)
brasileirao = brasileirao[["Cl", "shield", "Equipe", "JG", "V", "E", "D", "GP", "GC", "SG", "Pts", "Pts/PPJ", "xG", "xGC", "xSG", "xSG/90"]]


In [ ]:
# Define colors and fonts for table
bg_color = "#FFFAFA"
text_color = "#000000"

row_colors = {
    "Libertadores": "#9ACD32",
    "Pré-Libertadores": "#FFDEAD",
    "Rebaixamento": "#FF6347",
    "Sulamericana": "#7FFFD4",
    "Nada": "#F8F8FF",
}
plt.rcParams["text.color"] = text_color
font_normal = FontManager("https://github.com/google/fonts/raw/main/ofl/roboto/Roboto%5Bwdth,wght%5D.ttf")
font_bold = FontManager("https://github.com/google/fonts/raw/main/ofl/roboto/Roboto-Italic%5Bwdth,wght%5D.ttf")

In [ ]:
# Apply color to coluns
col_defs = [
    ColumnDefinition(
        name="Cl",
        textprops={"ha": "center"},
        width=0.5,
    ),
    ColumnDefinition(
        name="shield",
        textprops={"ha": "center", "va": "center", 'color': bg_color},
        width=0.5,
        plot_fn=lambda ax, img: ax.imshow(img),
        #plot_fn=image,
    ),
    ColumnDefinition(
        name="Equipe",
        textprops={"ha": "left", "weight": "bold"},
        width=1.75,
    ),
    ColumnDefinition(
        name="JG",
        group="Partidas Disputadas",
        textprops={"ha": "center"},
        width=0.5,
    ),
    ColumnDefinition(
        name="V",
        group="Partidas Disputadas",
        textprops={"ha": "center"},
        width=0.5,
    ),
    ColumnDefinition(
        name="E",
        group="Partidas Disputadas",
        textprops={"ha": "center"},
        width=0.5,
    ),
    ColumnDefinition(
        name="D",
        group="Partidas Disputadas",
        textprops={"ha": "center"},
        width=0.5,
    ),
    ColumnDefinition(
        name="GP",
        group="Gols",
        textprops={"ha": "center"},
        width=0.5,
    ),
    ColumnDefinition(
        name="GC",
        group="Gols",
        textprops={"ha": "center"},
        width=0.5,
    ),
    ColumnDefinition(
        name="SG",
        group="Gols",
        textprops={"ha": "center"},
        width=0.5,
    ),
    ColumnDefinition(
        name="Pts",
        group="Pontos",
        textprops={"ha": "center"},
        width=0.5,
    ),
    ColumnDefinition(
        name="Pts/PPJ",
        group="Pontos",
        textprops={"ha": "center"},
        width=0.5,
    ),
    ColumnDefinition(
        name="xG",
        group="Gols esperados",
        textprops={"ha": "center", "color": "#000000", "weight": "bold", "bbox": {"boxstyle": "circle", "pad": 0.35}},
        cmap=normed_cmap(brasileirao["xG"], cmap=matplotlib.cm.PiYG, num_stds=2)
    ),
    ColumnDefinition(
        name="xGC",
        group="Gols esperados",
        textprops={"ha": "center", "color": "#000000", "weight": "bold", "bbox": {"boxstyle": "circle", "pad": 0.35}},
        cmap=normed_cmap(brasileirao["xGC"], cmap=matplotlib.cm.PiYG_r, num_stds=2)
    ),
    ColumnDefinition(
        name="xSG",
        group="Gols esperados",
        textprops={"ha": "center", "color": "#000000", "weight": "bold", "bbox": {"boxstyle": "circle", "pad": 0.35}},
        cmap=normed_cmap(brasileirao["xSG"], cmap=matplotlib.cm.PiYG, num_stds=2)
    ),
]

In [ ]:
# Load shield from championship
shield_br = Image.open(
    urlopen("https://tmssl.akamaized.net/images/logo/header/bra1.png")
)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 22))
fig.set_facecolor(bg_color)
ax.set_facecolor(bg_color)
table = Table(
    brasileirao,
    column_definitions=col_defs,
    index_col="Cl",
    row_dividers=True,
    row_divider_kw={"linewidth": 1, "linestyle": (0, (1, 5))},
    footer_divider=True,
    textprops={"fontsize": 14},
    col_label_divider_kw={"linewidth": 1, "linestyle": "-"},
    column_border_kw={"linewidth": .5, "linestyle": "-"},
    ax=ax,
).autoset_fontcolors(colnames=["xG", "xGC", "xSG"]) 

table.cells[10, 3].textprops["color"] = "#8ACB88"


TITLE_BR = "Brasileirão Serie A | Temporada 2023"
SUB_TITLE = "38ª Rodada"
SIGNATURE = "       ericsonmatheus           ericson-matheus"

# Add title table
fig.text(0.21, 0.95, TITLE_BR, size=29, color="black", fontproperties=font_bold.prop)
# Add sub title table
fig_text(
    0.21, 0.94, SUB_TITLE, color="black",
    size=25, fig=fig, fontproperties=font_bold.prop
)
# Add signatures
fig_text(
    0.21, 0.91, SIGNATURE, color="black", size=25, fontproperties=font_bold.prop
)
# Add championship's shield to fig
fig.figimage(shield_br, xo=25, yo=1710, origin='upper', zorder=10)


github_icon_path = "../images/github.png"
github_icon = Image.open(github_icon_path)
n_size = (50,50)
github_icon = github_icon.resize(n_size)
fig.figimage(github_icon, xo=180, yo=1730, origin='upper')


linkedin_icon_path = "../images/linkedin.png"
linkedin_icon = Image.open(linkedin_icon_path)
n_size = (50,50)
linkedin_icon = linkedin_icon.resize(n_size)
fig.figimage(linkedin_icon, xo=520, yo=1730, origin='upper')


for idx in [0, 1, 2, 3, 6, 10]:
    table.rows[idx].set_facecolor(row_colors["Libertadores"])

for idx in [4, 5]:
    table.rows[idx].set_facecolor(row_colors["Pré-Libertadores"])

for idx in [7,8,9,11,12,13]:
    table.rows[idx].set_facecolor(row_colors["Sulamericana"])    

for idx in [16,17, 18, 19]:
    table.rows[idx].set_facecolor(row_colors["Rebaixamento"])

fig.savefig(
    "../layers/3.output/brasileirão-2023.png",
    facecolor=ax.get_facecolor(),
    dpi=100,
    bbox_inches="tight",
)
fig.show()